In [92]:
# Remove comments if you need to install any of the following packages
# Pip install big query for pulling publically available data via python
pip install google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Install a needed dependency for the .to_dataframe() function 
pip install pyarrow

In [3]:
# Engine helps pandas to process excel sheets / workbooks
pip install openpyxl

  Using cached openpyxl-3.0.7-py2.py3-none-any.whl (243 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [95]:
# This is being used to access a downloaded csv. 
# Import the required libraries.
import pandas as pd
import numpy as np

# only import bigquery from google cloud
from google.cloud import bigquery

In [21]:
def colManip(colVal):
    """Used with apply in pandas dataframe to specifically split any value in the column that has
    an '=' in anywhere in the string
    
    Inputs: colVal(str)
    
    Return: colVal(str) when value is NaN
            colVal(dict) otherwise"""
    try:
        if np.isnan(colVal) == True:
                return colVal
    except:
        colVal = colVal.split('\n')
        colVal = {val.split('=')[0]: val.split('=')[1] if '=' in val else val for val in colVal}
        return colVal
    
def encoding(colVal):
    """Encodes column values with apply in pd.dataframe
    Inputs: colVal(str)
    Returns: None"""
    
    return None

In [22]:
# Importing the transportation survey from the San Francisco Municipality Transportation... 
# The data dictionary will be the only sheet imported to DataDictionary.
DataDictionary = pd.read_excel('~/SanFranciscoBikeShare/TDS_2017_Data_WEBPAGE.xlsx', sheet_name='Data Dictionary',
              engine='openpyxl', usecols=range(4))

# Use pd.concat to pick the dataframes that meet our criteria and join them together.
# This criterion was selected in excel. The original highlighted sheet is available in the data folder.
DataDictionary = pd.concat([DataDictionary[DataDictionary['Description'].str.contains('Bay Area') == True],
                            DataDictionary[DataDictionary['Field'].str.contains('BIKE') == True],
                            DataDictionary[DataDictionary['Description'].str.contains('Zip code') == True],
                            DataDictionary[DataDictionary['Description'].str.contains('Code') == True]])

# Changes the Codes type from string to dictionary that can be used in processing the survey data.
DataDictionary.Codes = DataDictionary.Codes.apply(colManip)

In [23]:
DataDictionary

,Field,Field Type,Description,Codes
1,Q2,Integer,In which Bay Area county do you live?,"{'1': 'Alameda', '2': 'Contra Costa', '3': 'Ma..."
3,Trips,Integer,Total trips taken- SF residents: Sum of Q4Tot+...,NaN
54,BIKE,Integer,Total of all Bicycle (9) entries for Q4aM to Q5hM,NaN
126,BIKE1,Integer,Total of all Bicycle (9) entries for Day 1 (Q7...,NaN
138,BIKE2,Integer,Total of all Bicycle (9) entries for Day 2 (Q1...,NaN
180,Zone,Integer,(San Francisco Residents) San Francisco Zone (...,"{'1': 'One', '2': 'Two', '3': 'Three', '4': 'F..."
165,Q25A-SF,Integer,(San Francisco Residents) What is your home ZI...,"{'1': '94102 ', '2': '94103 ', '3': '94104 ', ..."


In [24]:
# Using the Field column in the DataDictionary dataframe as a list, can help select the columns in the actual survey
inputList = DataDictionary.Field.str.replace('Zone','ZONE').to_list()

# Imports the Data page from the same excel workbook mentioned previous. This is the survey data shorted to the
# Scope of columns selected in the DataDictionary
Survey2017 = pd.read_excel('~/SanFranciscoBikeShare/TDS_2017_Data_WEBPAGE.xlsx', sheet_name='Data',
              engine='openpyxl', usecols=inputList)

# Preview of the survey before manipulation
Survey2017.head()

,Q2,Trips,BIKE,BIKE1,BIKE2,Q25A-SF,ZONE
0,1,19,NaN,0.0,0.0,NaN,NaN
1,9,4,0.0,NaN,NaN,1.0,2.0
2,9,9,0.0,NaN,NaN,8.0,1.0
3,7,33,NaN,0.0,0.0,NaN,NaN
4,9,7,0.0,NaN,NaN,2.0,1.0


In [7]:
Survey2017.describe()

,Q2,Trips,BIKE,BIKE1,BIKE2,Q25A-SF,ZONE
count,804.000000,804.000000,401.000000,403.000000,403.000000,401.000000,401.000000
mean,6.284826,15.095771,0.122195,0.049628,0.007444,14.164589,2.947631
std,3.097377,26.272393,0.665230,0.996271,0.149441,7.814592,1.640046
min,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,3.000000,4.000000,0.000000,0.000000,0.000000,8.000000,2.000000
50%,8.000000,6.000000,0.000000,0.000000,0.000000,14.000000,3.000000
75%,9.000000,11.000000,0.000000,0.000000,0.000000,19.000000,4.000000
max,9.000000,330.000000,8.000000,20.000000,3.000000,30.000000,9.000000


Above, we can see that there are about 804 participants even though some might have not answered all the questions.

In [67]:
# Using the pre processed information from the DD in the Codes col, we can tel how many participants
# from each San Francisco Bay Area county these participants are coming from
codes = [code for code in DataDictionary.Codes]
q2dict = codes[0]
q2dict.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', ''])

In [46]:
# How many people are in this survey from which county? Are there any people who are out of town?
county_count = Survey2017.groupby('Q2')['Trips'].count().reset_index()
county_count.columns = ['Q2','NumResidents']
county_count.Q2 = county_count.Q2.apply(lambda x: q2dict[str(x)] if str(x) in q2dict.keys() else x)
county_count

,Q2,NumResidents
0,Alameda,87
1,Contra Costa,83
2,Marin,49
3,Napa,6
4,San Mateo,101
5,Santa Clara,43
6,Solano,15
7,Sonoma,19
8,San Francisco,401


In [88]:
# How many people are in this survey from which county? Are there any people who are out of town?
zipCodes = Survey2017.groupby('Q25A-SF').agg({'Trips':['count','sum']})
zipCodes.columns = pd.MultiIndex.from_arrays([['Trips', 'Trips'],['Responses','sum']])
zipCodes.index = [codes[-1][str(int(zC))] if str(int(zC)) in codes[-1] else zC for zC in zipCodes.index.values]
zipCodes

Trips     
         Responses  sum
94102           23  135
94103           17   98
94105            6   38
94107            8   38
94108            9   32
94109           19  101
94110           32  177
94111            1   13
94112           28  157
94114           19  146
94115           15   79
94116           20  139
94117           18  129
94118           19  117
94121           22  118
94122           23  108
94123            8   53
94124           18  116
94127           12   65
94129            5   35
94131           21   95
94132           12   82
94133           11   66
94134           24  153
94158            3   20
Refused          8   44

In [89]:
# Now that we have the codes in place from the survey, let's see if it connects to any of the zip codes
joinZips = codes[-1]

In [93]:
# Changing the settings to override this file path in the environment
import os

# Needing to have the key file downloaded to access via the api. The query can be used
# instead and utilized for this brief analysis.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/tiareina/Downloads/compact-scene-317315-b4b3cd5c140c.json"

In [96]:
# Construct a BigQuery client object.
client = bigquery.Client()

query = """
    WITH distinctZips AS

    (SELECT DISTINCT zip_code
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE LENGTH(zip_code) >= 5)

    SELECT split(zip_code, '-') 
    FROM distinctZips
    WHERE LENGTH(zip_code) = 5;
"""

query_job = client.query(query)  # Make an API request.

In [138]:
bQzips = query_job.to_dataframe()

In [139]:
bQzips.columns = ['ZipCode']
bQzips.ZipCode = bQzips.ZipCode.apply(lambda x: x[0])
bQzips.ZipCode=bQzips.ZipCode.apply(lambda x: np.NaN if x[0] not in '0123456789' else x)
bQzips.sort_values('ZipCode').dropna()

,ZipCode
2891,10000
962,10001
1330,10002
734,10003
699,10004
...,...
5397,99867
5887,99901
1527,99954
5174,99958


Interestingly enough we have New Yorker riders in the San Francisco data. We could possibly add this to our analysis.

In [141]:
pd.merge(bQzips, zipCodes, left_on='ZipCode', right_on=zipCodes.index)

,ZipCode,"(Trips, Responses)","(Trips, sum)"
0,94110,32,177
1,94107,8,38
2,94115,15,79
3,94118,19,117
4,94131,21,95
5,94133,11,66
6,94158,3,20
7,94117,18,129
8,94111,1,13
9,94112,28,157


In [142]:
len(zipCodes)

26